In [75]:
from qiskit import *
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import minimize
df = pd.read_csv (r'Data.csv')

In [76]:
Y = list(df['Y'])
Theta = list(df['Theta'])
State = list(df['State'])

In [90]:
Y_Train = Y[:1000]
T_Train = Theta[:1000]
S_Train = State[:1000]

Y_Test = Y[1000:]
T_Test = Theta[1000:]
S_Test = State[1000:]

In [92]:
df_1 = pd.DataFrame({'Y':Y_Train, 'Theta':T_Train,'State':S_Train,'z': [0]*1000})
fig = px.scatter_3d(df_1, x='Y', y='Theta',z='z', color='State')
fig.show()

In [93]:
def Circuit_v1(Params,V):
    qc = QuantumCircuit(2,2)
    qc.h(range(2))
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.cx(0,1)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),1)
    qc.cx(0,1)
    for i in range(int(len(Params)/3)):
        qc.ry(Params[0+3*i],0)
        qc.ry(Params[1+3*i],1)
        qc.cry(Params[2+3*i],0,1)
    qc.measure([0,1],[0,1])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=500)
    job_result = job.result()
    counts = job_result.get_counts(qc)
    Keymax = max(counts, key=counts.get) 
    return (int(Keymax,2))
    
        

In [94]:
Circuit_v1([0,0,0],[0,0])

1

In [95]:
def Circuit_v2(Params,V):
    qc.QuantumCirucit()

In [96]:
Params = [0.5]*9

In [97]:
def Cost(Params):
    cost = 0
    for i in range (len(Y_Train)):
        if Circuit_v1(Params,[Y_Train[i],T_Train[i]]) != S_Train[i]:
            cost = cost+100
    return(cost)

In [98]:
1- Cost(Params)/50000

-0.262

In [99]:
minimum = minimize(Cost, Params, method="COBYLA", tol=0.001)

In [100]:
opt_params = minimum.x

In [101]:
def eff(Params):
    cost = 0
    for i in range (len(Y_Test)):
        if Circuit_v1(Params,[Y_Test[i],T_Test[i]]) != S_Test[i]:
            cost = cost+100
    return(1-(cost/(100*len(Y_Test))))

In [102]:
eff(opt_params)

0.556

In [11]:
#! /usr/bin/python3

import sys
import pennylane as qml
import numpy as np
from scipy.optimize import minimize
import math

def classify_data(X_train, Y_train, X_test):
    #print("sdfsdf")
    """Develop and train your very own variational quantum classifier.

    Use the provided training data to train your classifier. The code you write
    for this challenge should be completely contained within this function
    between the # QHACK # comment markers. The number of qubits, choice of
    variational ansatz, cost function, and optimization method are all to be
    developed by you in this function.

    Args:
        X_train (np.ndarray): An array of floats of size (250, 3) to be used as training data.
        Y_train (np.ndarray): An array of size (250,) which are the categorical labels
            associated to the training data. The categories are labeled by -1, 0, and 1.
        X_test (np.ndarray): An array of floats of (50, 3) to serve as testing data.

    Returns:
        str: The predicted categories of X_test, converted from a list of ints to a
            comma-separated string.
    """

    # Use this array to make a prediction for the labels of the data in X_test
    predictions = []
    # QHACK #
    dev = qml.device("default.qubit", wires=3, shots=1)
    @qml.qnode(dev)
    def circuit1(data,params):
        for i in range(3):
            qml.Hadamard(wires = i)
            qml.RY(data[i],wires = i)



        def entangling_layer(nqubits):

            for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
                qml.CNOT(wires=[i, i + 1])
            for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
                qml.CNOT(wires=[i, i + 1])
            '''
        qml.CZ(wires=[0,1])
        qml.RY((data[0]+data[1])/2,wires = 1)
        qml.CZ(wires=[0,1])
        qml.CZ(wires=[0,2])
        qml.RY((data[0]+data[2])/2,wires = 2)
        qml.CZ(wires=[0,2])
        qml.CZ(wires=[2,1])
        qml.RY((data[2]+data[1])/2,wires = 1)
        qml.CZ(wires=[2,1])
        '''
        #parameters = params.reshape((2, 3, 3))
        #qml.templates.StronglyEntanglingLayers(parameters, wires=range(3))
        #entangling_layer(3)
        '''
        for j in range(3):
            qml.RY(params[j],wires = j)
        '''
        for i in range(1):
            for j in range(3):
                qml.RY(params[i*3+j],wires = j)
                
            qml.CNOT(wires = [0,1])
            qml.CNOT(wires = [0,2])
            qml.CNOT(wires = [1,2])
            qml.CNOT(wires = [2,0])
            for j in range(3):
                qml.RY(params[i*3+3+j],wires = j)
            qml.CNOT(wires = [0,1])
            qml.CNOT(wires = [0,2])
            qml.CNOT(wires = [1,2])
            qml.CNOT(wires = [2,0])

        '''
        qml.CRY(params[0],wires = [0,1])
        qml.CRY(params[1],wires = [0,2])
        qml.CRY(params[2],wires = [1,2])
        qml.CRY(params[3],wires = [2,0])
        qml.CRY(params[4],wires = [0,1])
        qml.CRY(params[5],wires = [1,0])
        qml.CRY(params[6],wires = [1,2])
        qml.CRY(params[7],wires = [2,0])
        '''
        return qml.expval(qml.PauliZ(0) )#@ qml.PauliZ(1))
        #return qml.probs(wires=(range(3)))
    params = [0,0,0,0,0,0]
    # QHACK #
    #print( Y_train)
    def pred(data,params):
        z = circuit1(data,params)
        '''
        #print(z)
        x = 0
        y= 0
        
        
        for i in range (2**3):
            if z[i]>y:
                y = z[i]
                x = i
        #print(x)
        
        if x<3:
            x = -1
        elif 2<x<6:
            x = 0
        else:
            x = 1
        
        return x
        
        '''
        z = math.floor(z*3-1)
        return z
    
    def cost(params):
        cost = 0
        for i in range (250):
            z = math.floor(circuit1(X_train[i],params)*3-1)
            #o = pred(X_train[i],params)
            #print(pred(X_train[i],params))
            if z != Y_train[i]:
                cost +=100
        return cost
    
    #print(predictions)
    #print(cost(params))
    '''
    opt = qml.AdagradOptimizer(stepsize=0.5)
    steps = 30
    for i in range(steps):
        params = opt.step(cost, params)
    '''
    minimum = minimize(cost, params, method="COBYLA", tol=0.01)
    #print(minimum.x)
    opt_params = minimum.x
    for i in range (50):
        predictions.append(pred(X_test[i],opt_params))
    
    

    #print(cost(opt_params))
    #print(opt_params)
    Try = [1,0,-1,0,-1,1,-1,-1,0,-1,1,-1,0,1,0,-1,-1,0,0,1,1,0,-1,0,0,-1,0,-1,0,0,1,1,-1,-1,-1,0,-1,0,1,0,-1,1,1,0,-1,-1,-1,-1,0,0]
    a =0
    
    for i in range(50):
        if Try[i]==predictions[i]:
            a = a+1
    
    #print(a)
    #print(predictions==Try)
    #print(cost(opt_params))
    return array_to_concatenated_string(predictions)


def array_to_concatenated_string(array):
    """DO NOT MODIFY THIS FUNCTION.

    Turns an array of integers into a concatenated string of integers
    separated by commas. (Inverse of concatenated_string_to_array).
    """
    return ",".join(str(x) for x in array)


def concatenated_string_to_array(string):
    """DO NOT MODIFY THIS FUNCTION.

    Turns a concatenated string of integers separated by commas into
    an array of integers. (Inverse of array_to_concatenated_string).
    """
    return np.array([int(x) for x in string.split(",")])


def parse_input(giant_string):
    """DO NOT MODIFY THIS FUNCTION.

    Parse the input data into 3 arrays: the training data, training labels,
    and testing data.

    Dimensions of the input data are:
      - X_train: (250, 3)
      - Y_train: (250,)
      - X_test:  (50, 3)
    """
    X_train_part, Y_train_part, X_test_part = giant_string.split("XXX")

    X_train_row_strings = X_train_part.split("S")
    X_train_rows = [[float(x) for x in row.split(",")] for row in X_train_row_strings]
    X_train = np.array(X_train_rows)

    Y_train = concatenated_string_to_array(Y_train_part)

    X_test_row_strings = X_test_part.split("S")
    X_test_rows = [[float(x) for x in row.split(",")] for row in X_test_row_strings]
    X_test = np.array(X_test_rows)

    return X_train, Y_train, X_test


if __name__ == "__main__":
    # DO NOT MODIFY anything in this code block

    X_train, Y_train, X_test = parse_input(sys.stdin.read())
    output_string = classify_data(X_train, Y_train, X_test)
    print(f"{output_string}")


ValueError: not enough values to unpack (expected 3, got 1)